# 动态ONNX推理量化测试

## 1. FP32精度测试 

In [1]:
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

def preprocess(image):
    channel_num = image.layers  #通道数

    # Resize
    ratio = 800.0 / min(image.size[0], image.size[1])
    image = image.resize((int(ratio * image.size[0]), int(ratio * image.size[1])),  Image.Resampling.BILINEAR)
    
    if channel_num == 1:  #灰度图像转三通道
        # print("trans")
        image = cv2.cvtColor(np.array(image) , cv2.COLOR_GRAY2RGB)
   
    # Convert to BGR
    image = np.array(image)[:, :, [2, 1, 0]].astype('float32')

    # HWC -> CHW
    image = np.transpose(image, [2, 0, 1])

    # Normalize
    mean_vec = np.array([102.9801, 115.9465, 122.7717])
    for i in range(image.shape[0]):
        image[i, :, :] = image[i, :, :] - mean_vec[i]

    # Pad to be divisible of 32
    import math
    padded_h = int(math.ceil(image.shape[1] / 32) * 32)
    padded_w = int(math.ceil(image.shape[2] / 32) * 32)

    padded_image = np.zeros((3, padded_h, padded_w), dtype=np.float32)
    padded_image[:, :image.shape[1], :image.shape[2]] = image
    image = padded_image

    return image


In [3]:
with open("./images_list.txt","r") as f:
   img_path_list =  f.readlines()
img_path_list  = [x.rstrip() for x in img_path_list]
# img_path_list  #数据集路径

In [3]:
img = Image.open(img_path_list[0])
img.size[::-1]

(427, 640)

In [5]:
height = []
width = []
for path in tqdm(img_path_list):
    img = Image.open(path)
    input_tensor = preprocess(img)
    height.append(input_tensor.shape[1])
    width.append(input_tensor.shape[2])

100%|██████████| 4952/4952 [02:39<00:00, 30.95it/s]


In [7]:
import pandas as pd
df = pd.DataFrame({"height":height,"width":width})
df.describe()

,height,width
count,4952.000000,4952.000000
mean,884.025848,1067.825525
std,158.063905,208.652261
min,800.000000,800.000000
25%,800.000000,800.000000
50%,800.000000,1088.000000
75%,800.000000,1216.000000
max,2112.000000,3232.000000


In [3]:
import sys
sys.path.append("../detection")
from utils import generate_calib_data
from torch.utils.data import DataLoader
from dataset import build_dataset
from mmcv.parallel import collate


# 数据配置文件
ann_file = "/home/geng/fiftyone/coco-2017/validation/labels.json"
data_root = '/home/geng/fiftyone/coco-2017/validation/data/'  # 数据的根路径。
batch_size = 1
input_size = (480,640)  #或者 (1,3,800,1216)


# 数据加载
dataset = build_dataset(ann_file=ann_file,data_root=data_root,input_size=input_size,keep_ratio=False)
dataloader = DataLoader(dataset,batch_size=batch_size,collate_fn=collate)

generate_calib_data(dataset=dataset,seed_num=0,calib_num=512)

loading annotations into memory...
Done (t=0.84s)
creating index...
index created!


In [ ]:
from 

In [10]:
import onnxruntime
import os

# ort_custom_op_path = "/home/geng/tinyml/mmdeploy/build/lib/libmmdeploy_onnxruntime_ops.so"
# session_options = onnxruntime.SessionOptions()
# session_options.register_custom_ops_library(ort_custom_op_path)

onnxruntime_model_path = "/home/geng/tinyml/ppq/benchmark/detection/dynamic_input_test/FasterRCNN-NMS-FP32.onnx"
providers = ['CUDAExecutionProvider']  #
sess = onnxruntime.InferenceSession(path_or_bytes=onnxruntime_model_path, providers=providers)
input_placeholder_name = sess.get_inputs()[0].name

outputs = []
for img_path in tqdm(img_path_list):
    img = Image.open(img_path)
    try:
        input_tensor = preprocess(img)
    except IndexError:
        print(f"error happend in precess {img_path}")
        break
    output = sess.run(input_feed={input_placeholder_name: input_tensor}, output_names=None)
    outputs.append({"file_name":img_path,"img_size":img.size,"output":output})
    break

Fail: [ONNXRuntimeError] : 1 : FAIL : Load model from /home/geng/tinyml/ppq/benchmark/detection/dynamic_input_test/FasterRCNN-NMS-FP32.onnx failed:Fatal error: TRTBatchedNMS is not a registered function/op

In [10]:
import os
os.getcwd()
os.chdir("..")

In [7]:
# 测试实际检测效果
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from dataset import coco_classes

# idx = 0
# classes = coco_classes()

# def display_objdetect_image(image, boxes, labels, scores, score_threshold=0.7):
#     # Resize boxes
#     ratio = 800.0 / min(image.size[0], image.size[1])
#     boxes /= ratio

#     _, ax = plt.subplots(1, figsize=(12,9))
#     image = np.array(image)
#     ax.imshow(image)

#     # Showing boxes with score > 0.7
#     for box, label, score in zip(boxes, labels, scores):
#         if score > score_threshold:
#             rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=1, edgecolor='b', facecolor='none')
#             ax.annotate(classes[label-1] + ':' + str(np.round(score, 2)), (box[0], box[1]), color='w', fontsize=12)
#             ax.add_patch(rect)
#     plt.show()
# img = Image.open(img_path_list[idx])
# display_objdetect_image(img, *outputs[idx]["output"])

In [3]:
# 测试分割结果
import sys
sys.path.append("../detection/")

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from dataset import coco_classes
import pycocotools.mask as mask_util
import cv2


idx = 0
classes = coco_classes()
tmp = None
def display_objdetect_image(image, boxes, labels, scores, masks, score_threshold=0.7):
    # Resize boxes
    ratio = 800.0 / min(image.size[0], image.size[1])
    boxes /= ratio

    _, ax = plt.subplots(1, figsize=(12,9))

    image = np.array(image)

    for mask, box, label, score in zip(masks, boxes, labels, scores):
        # Showing boxes with score > 0.7
        if score <= score_threshold:
            continue
        
        # Finding contour based on mask
        mask = mask[0, :, :, None]
        int_box = [int(i) for i in box]
        mask = cv2.resize(mask, (int_box[2]-int_box[0]+1, int_box[3]-int_box[1]+1))
        mask = mask > 0.5
        im_mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        x_0 = max(int_box[0], 0)
        x_1 = min(int_box[2] + 1, image.shape[1])
        y_0 = max(int_box[1], 0)
        y_1 = min(int_box[3] + 1, image.shape[0])
        mask_y_0 = max(y_0 - box[1], 0)
        mask_y_1 = mask_y_0 + y_1 - y_0
        mask_x_0 = max(x_0 - box[0], 0)
        mask_x_1 = mask_x_0 + x_1 - x_0
        im_mask[y_0:y_1, x_0:x_1] = mask[
            mask_y_0 : mask_y_1, mask_x_0 : mask_x_1
        ]
        im_mask = im_mask[:, :, None]
        print(im_mask.shape)

        # OpenCV version 4.x
        contours, hierarchy = cv2.findContours(
            im_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
        )

        image = cv2.drawContours(image, contours, -1, 25, 3)

        rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=1, edgecolor='b', facecolor='none')
        ax.annotate(classes[label-1] + ':' + str(np.round(score, 2)), (box[0], box[1]), color='w', fontsize=12)
        ax.add_patch(rect)

    ax.imshow(image)
    plt.show()
    
img = Image.open(img_path_list[idx])
display_objdetect_image(img,  *outputs[idx]["output"])

NameError: name 'Image' is not defined

In [7]:
import numpy as np
from itertools import groupby
def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle




# rle = binary_mask_to_rle(np.squeeze(tmp))

In [12]:
img = Image.open(img_path_list[0])
img.size

(640, 427)

In [5]:
# import torch
# class_num = 80

# # 目标检测格式化
# results = []
# for output in tqdm(outputs):
#     bboxs,labels,scores = output["output"]
#     result = [[] for _ in range(class_num)]
#     ratio = 800.0 / min(output["img_size"][0], output["img_size"][1])
#     bboxs /= ratio
    
#     bboxs = torch.cat((torch.from_numpy(bboxs),torch.from_numpy(scores.reshape((-1,1)))),dim=1)
#     for i,label in enumerate(labels):
#         result[label-1].append(bboxs[i])
#     results.append(result)

100%|██████████| 4952/4952 [00:02<00:00, 2184.16it/s]


In [11]:
bboxs,labels,scores,masks = outputs[0]["output"]

In [24]:
mask = masks[0, :, :, None]
mask.shape

(1, 28, 1, 28)

In [21]:
masks.shape

(100, 1, 28, 28)

In [7]:
import torch
class_num = 80

import  pycocotools.mask as mk



# 实例分割格式化
results = []
for output in tqdm(outputs): #遍历每张图片

    bboxs,labels,scores,masks = output["output"]  # 每个图片有若干个bbox
    result_det = [[] for _ in range(class_num)]
    result_seg = [[] for _ in range(class_num)]

    ratio = 800.0 / min(output["img_size"][0], output["img_size"][1])
    bboxs /= ratio   

    for mask, box, label in zip(masks, bboxs, labels):
        mask = mask[0, :, :, None]
        int_box = [int(i) for i in box]
        mask = cv2.resize(mask, (int_box[2]-int_box[0]+1, int_box[3]-int_box[1]+1))
        mask = mask > 0.5
        
        im_mask = np.zeros((output["img_size"][0], output["img_size"][1]), dtype=np.uint8)
        print(mask.shape,im_mask.shape)
        x_0 = max(int_box[0], 0)
        x_1 = min(int_box[2] + 1, output["img_size"][1])
        y_0 = max(int_box[1], 0)
        y_1 = min(int_box[3] + 1, output["img_size"][0])
        mask_y_0 = max(y_0 - box[1], 0)
        mask_y_1 = mask_y_0 + y_1 - y_0
        mask_x_0 = max(x_0 - box[0], 0)
        mask_x_1 = mask_x_0 + x_1 - x_0
        im_mask[y_0:y_1, x_0:x_1] = mask[mask_y_0 : mask_y_1, mask_x_0 : mask_x_1]  #原图尺寸的掩膜
        
        rle = mk.encode(np.asfortranarray(im_mask))
        print(rle)

        result_seg[label-1].append(rle)
    
    bboxs = torch.cat((torch.from_numpy(bboxs),torch.from_numpy(scores.reshape((-1,1)))),dim=1)
    for i,label in enumerate(labels):
        result_det[label-1].append(bboxs[i])
        
    results.append((result_det,result_seg))

  0%|          | 0/1 [00:00<?, ?it/s]

(154, 63) (640, 427)


ValueError: ndarray is not Fortran contiguous

In [10]:
outputs[0]

{'file_name': '/home/geng/fiftyone/coco-2017/validation/data/000000397133.jpg',
 'img_size': (640, 427),
 'output': [array([[2.04250473e+02, 3.60535049e+01, 2.66997101e+02, 1.89642776e+02],
         [3.26094836e-01, 1.38050232e+02, 3.13769264e+01, 1.63540085e+02],
         [2.77355835e+02, 2.53986130e+01, 2.84665771e+02, 5.86160355e+01],
         [1.62398510e+01, 1.84005142e+02, 5.26393051e+01, 2.03903519e+02],
         [8.20932465e+01, 8.98702316e+01, 9.72104874e+01, 9.85564651e+01],
         [2.72977051e+02, 2.29581413e+01, 2.78524811e+02, 4.82035217e+01],
         [3.07047482e+01, 1.53708328e+02, 7.28398743e+01, 1.74533569e+02],
         [2.85055511e+02, 1.88136978e+01, 2.94541046e+02, 6.97207413e+01],
         [1.24394539e+02, 9.89613647e+01, 2.09066238e+02, 1.72448273e+02],
         [7.58380737e+01, 1.43180710e+02, 9.19904556e+01, 1.61016800e+02],
         [2.92728973e+02, 2.34075947e+01, 3.02066040e+02, 5.92476692e+01],
         [2.60880280e+02, 1.08030556e+02, 3.28447968e+02, 1.

In [8]:
from ..dataset import build_dataset
ann_file = "/home/geng/fiftyone/coco-2017/validation/labels.json"
data_root = '/home/geng/fiftyone/coco-2017/validation/data/'  # 数据的根路径。
batch_size = 1
input_size = (480,640)  #或者 (1,3,480,640)

# 数据加载
dataset = build_dataset(ann_file=ann_file,data_root=data_root,input_size=input_size)
dataset.results2json(results=results,outfile_prefix="/home/geng/tinyml/ppq/benchmark/detection/FP32_model/FasterRCNN-12-FP32") #也可以将推理结果持久化

ImportError: attempted relative import with no known parent package

In [7]:
# dataset.evaluate(results=results) 
dataset.evaluate(results_json_path="/home/geng/tinyml/ppq/benchmark/detection/FP32_model/FasterRCNN-12-FP32.bbox.json") 


Evaluating bbox...
Loading and preparing results...
DONE (t=0.86s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=66.03s).
Accumulating evaluation results...
DONE (t=13.37s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.578
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.373
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.185
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.387
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.473
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.

OrderedDict([('bbox_mAP', 0.352),
             ('bbox_mAP_50', 0.578),
             ('bbox_mAP_75', 0.373),
             ('bbox_mAP_s', 0.185),
             ('bbox_mAP_m', 0.387),
             ('bbox_mAP_l', 0.473),
             ('bbox_mAP_copypaste', '0.352 0.578 0.373 0.185 0.387 0.473')])